In [269]:
"""
@DATE: 2021/10/17
@Author: Ziqi Wang
@File: bts.py
"""

import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def next_action(self, point):
        i, j = point
        if 0 <= i < self.n and 0 <= j < self.n and i * self.n + j < self.n ** 2 - 1:
            j += 1
            if j == self.n:
                j = 0
                i += 1
            return i, j
        else:
            return None

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

# BTS: Backtracking search

In [270]:
def bts(problem, start=(0, 0)):
    size = 8
    action_stack = []
    cnt = 0 # 列
    column = [0] * size
    while not problem.is_satisfy(action_stack):
        # print("action_stack = ", action_stack)
        # print("column = ", column) # 1 4 2 5 0 0
        # print("cnt = ", cnt)
        stack = action_stack.copy()
        
        stack.append((column[cnt], cnt))# 5 3
        if column[cnt] >= size:
            column[cnt] = 0
            cnt = cnt - 1
            action_stack.remove((column[cnt] - 1, cnt))
            continue
        # print("stack = ", stack)
        if problem.is_valid(stack):
            action_stack.append((column[cnt], cnt))
            column[cnt] = column[cnt] + 1
            cnt = cnt + 1
            # print("action_stack = ", action_stack)
        else:
            stack.remove((column[cnt], cnt))
            if column[cnt] == size - 1:
                column[cnt] = 0
                cnt = cnt - 1
                action_stack.remove((column[cnt] - 1, cnt))
            while column[cnt] < size - 1:
                column[cnt] = column[cnt] + 1
                stack.append((column[cnt], cnt))
                # print("In else, stack = ", stack)
                if problem.is_valid(stack):
                    action_stack.append((column[cnt], cnt))
                    column[cnt] = column[cnt] + 1
                    cnt = cnt + 1
                    break
                elif column[cnt] != size - 1:
                    stack.remove((column[cnt], cnt))
                elif column[cnt] >= size - 1:
                    column[cnt] = 0
                    cnt = cnt - 1
                    action_stack.remove((column[cnt] - 1, cnt))
                    # print("In size = 4, action_stack = ", action_stack)
                    break

                     
        yield action_stack

# Improving BTS
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [271]:
def get_maximum(column, cnt):
    maximum = -10000
    idx = -1
    for i in range(len(column)):
        if column[i] > maximum and i < cnt:
            maximum = column[i]
            idx = i
    return idx

        

def improving_bts(problem, start=(0, 0)):
    size = 16
    action_stack = []
    cnt = 0 # 列
    column = [0] * size
    while not problem.is_satisfy(action_stack):
        # print("action_stack = ", action_stack)
        # print("column = ", column) # 1 4 2 5 0 0
        # print("cnt = ", cnt)
        stack = action_stack.copy()
        
        stack.append((column[cnt], cnt))# 5 3
        if column[cnt] >= size:
            column[cnt] = 0
            cnt = cnt - 1
            action_stack.remove((column[cnt] - 1, cnt))
            continue
        # print("stack = ", stack)
        if problem.is_valid(stack):
            action_stack.append((column[cnt], cnt))
            column[cnt] = column[cnt] + 1
            cnt = cnt + 1
            # print("action_stack = ", action_stack)
        else:
            stack.remove((column[cnt], cnt))
            if column[cnt] == size:
                column[cnt] = 0
                cnt = get_maximum(column=column)
                action_stack.remove((column[cnt] - 1, cnt))

            if column[cnt] == size - 1:
                column[cnt] = 0
                
                cnt = cnt - 1
                action_stack.remove((column[cnt] - 1, cnt))
            while column[cnt] < size - 1:
                column[cnt] = column[cnt] + 1
                stack.append((column[cnt], cnt))
                # print("In else, stack = ", stack)
                if problem.is_valid(stack):
                    action_stack.append((column[cnt], cnt))
                    column[cnt] = column[cnt] + 1
                    cnt = cnt + 1
                    break
                elif column[cnt] != size - 1:
                    stack.remove((column[cnt], cnt))
                elif column[cnt] >= size - 1:
                    column[cnt] = 0
                    cnt = cnt - 1
                    action_stack.remove((column[cnt] - 1, cnt))
                    # print("In size = 4, action_stack = ", action_stack)
                    break
                else:
                    if column[cnt] == size:
                        column[cnt] = 0
                        cnt = get_maximum(column=column)
                        action_stack.remove((column[cnt] - 1, cnt))


                     
        yield action_stack

In [272]:
# test_block
n = 16
render = (n == 1)
method = improving_bts  # here to change your method; bts or improving_bts
p = Problem(n)
if render:
    import pygame
    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = method(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen.png')
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in method(p):
        pass
    print(time.time() - start_time)
    p.print_state(actions)

173.02725219726562
_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
---------------------------------
